In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import sys
sys.path.append('../')
from mrcnn.utils import trim_zeros_graph
import numpy as np
import tensorflow as tf

np.set_printoptions(linewidth=100, precision=4)
sess = tf.InteractiveSession()

batch_size = 3
num_rois = 32
num_classes = 4
num_detections = 100
num_rois_per_image = 32

D:\Program Files\Anaconda3\envs\TF_gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


###  Stack non-zero bbox infromaton from pred_array into stacked_array

In [74]:
rois_per_image = 32
# ps = pred_tensor[~tf.all(pred_tensor[:,2:6] == 0, axis=1)]
print(' Pred tensor shape is : ', pred_tensor.shape)

pred_tensor_2 = pred_tensor[:,:,:,1:]
print(' Pred tensor 2 shape is : ', pred_tensor_2.shape)
# print(pred_tensor_2[0,0].eval())
# print(pred_tensor_2[0,1].eval())
# print(pred_tensor_2[0,2].eval())
p= tf.reshape(pred_tensor_2, [batch_size, num_classes * rois_per_image ,6])
print(' shape of p.shape', p.shape)

print(p[0,33:65].eval())

p_reduced = tf.reduce_sum(tf.abs(pred_tensor_2), axis=-1)
print(' p_redcued shape ',p_reduced.shape)
print(p_reduced[0].eval())
# #
mask = tf.greater(p_reduced , 0)
print(' mask shape ', mask.get_shape())
# print(mask.eval())

ind  = tf.where(mask)
print(' pred_tensor_2 shape ',pred_tensor_2.get_shape())
print(' ind shape ',ind.get_shape())
print(ind.eval())


dense = tf.gather_nd(  pred_tensor_2, ind)
dense = tf.scatter_nd(   ind,pred_tensor_2,[batch_size,rois_per_image, 6])

print(' dense shape ',dense.get_shape())

print(dense.eval())
# print(dense[1].eval())
# print(dense[2].eval())
# print(dense[3].eval())

# stacked_tensor = tf.reshape(dense, [batch_size,rois_per_image, 6])
print(stacked_tensor.get_shape())
print(stacked_tensor.eval())


tf.equal()

 Pred tensor shape is :  (3, 4, 32, 7)
 Pred tensor 2 shape is :  (3, 4, 32, 6)
 shape of p.shape (3, 128, 6)
[[  0.9281 106.9807 118.7934 106.3466   7.4645   1.    ]
 [  0.9126  39.4174 116.8137  50.6049 100.7003   1.    ]
 [  0.8094  38.2599 103.6086  84.2878  61.6374   1.    ]
 [  0.591   19.4185  75.6502  42.8045  19.8604   1.    ]
 [  0.4301  23.823   55.0586  15.4241  31.1665   1.    ]
 [  0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.  

ValueError: The inner 0 dimensions of output.shape=[?,?,?] must match the inner 3 dimensions of updates.shape=[3,4,32,6]: Shapes must be equal rank, but are 0 and 3 for 'ScatterNd_9' (op: 'ScatterNd') with input shapes: [?,3], [3,4,32,6], [3].

###  Stack non-zero bbox infromaton from pred_array into stacked_array --- method 2 (obsolete)

In [425]:
# sep = tf.unstack(pred_tensor)
# print(len(sep))
# all_boxes = []
# for sub_sep in sep:
#     p_reduced = tf.reduce_sum(tf.abs(sub_sep), axis=-1)
#     print(' p_redcued shape ',p_reduced.shape)
# #     print(p_reduced.eval())
#     non_zeros = tf.cast(p_reduced, tf.bool)
#     print(' mask shape' ,non_zeros.shape) 
#     # print(non_zeros.eval())
#     boxes = tf.boolean_mask(sub_sep, non_zeros,axis = 0)
#     print(' boxes shape ',boxes.get_shape())
#     print(boxes.eval())

#     all_boxes.append(boxes)
# print(len(all_boxes))
# stacked_tensor = tf.stack(all_boxes)
# print('stacked shape',stacked_tensor.get_shape())

# p_reduced = tf.reduce_sum(tf.abs(pred_tensor), axis=-1)
# print(' p_redcued shape ',p_reduced.shape)
# print(p_reduced.eval())
# non_zeros = tf.cast(p_reduced, tf.bool)
# print(' mask shape' ,non_zeros.shape) 
# print(non_zeros.eval())
# boxes = tf.boolean_mask(pred_tensor, non_zeros,axis = 0)
# print(boxes.shape)
# print(boxes.eval())
# print(keepers[1].eval())

### Build coordinate meshgrid 
Build in the following dimensions to allow for tensorflow Multivariate Normal Dist
    
    width x height x batch_size x rois per image x 2

In [420]:
rois_per_image  = 32
img_h  = 8
img_w  = 16
X = tf.range(img_w, dtype=tf.int32)
Y = tf.range(img_h, dtype=tf.int32)
X, Y = tf.meshgrid(X, Y)
print( X.get_shape(), Y.get_shape())
print( ' X : \n',X.eval())
print( ' Y : \n',Y.eval())

# ## hear we repeat X and Y  batch_size x rois_per_image times
ones = tf.ones([batch_size, rois_per_image,1, 1], dtype = tf.int32)
print(' ones: ',ones.shape)                
# # ones = tf.expand_dims(ones,-1)
# print(' ones with exp dims ',ones.shape)

rep_X = ones * X
rep_Y = ones * Y 
print(' ones_exp * X', ones.shape, '*', X.shape, '= ',rep_X.shape)
# print(rep_X[0,0].eval())
# print()
# print(rep_X[2,15].eval())
print(' ones_exp * Y', ones.shape, '*', Y.shape, '= ',rep_Y.shape)
# print(rep_Y[0,0].eval())
# print()
# print(rep_Y[2,15].eval())

# # stack the X and Y grids 
bef_pos = tf.to_float(tf.stack([rep_X,rep_Y], axis = -1))
print(' before transpse ', bef_pos.get_shape())
pos = tf.transpose(pos,[2,3,0,1,4])
print(' after transpose ', pos.get_shape())

(8, 16) (8, 16)
 X : 
 [[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]]
 Y : 
 [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
 [3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
 [4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
 [5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]
 [6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6]
 [7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]]
 ones:  (3, 32, 1, 1)
 ones_exp * X (3, 32, 1, 1) * (8, 16) =  (3, 32, 8, 16)
 ones_exp * Y (3, 32, 1, 1) * (8, 16) =  (3, 32, 8, 16)
 before transpse  (3, 32, 8, 16, 2)
 after transpose  (8, 16, 3, 32, 2)


#### Another method for creating a meshgrid with the following shape
    image width x image height  x batch_size x num_rois_per_image x 2    (8 x 16 x 3 x 32 x 2)

In [429]:
rois_per_image  = 32
img_h  = 8
img_w  = 16
X = tf.range(img_w, dtype=tf.int32)
Y = tf.range(img_h, dtype=tf.int32)
X, Y = tf.meshgrid(X, Y)
print( X.get_shape(), Y.get_shape())
print( ' X : \n',X.eval())
print( ' Y : \n',Y.eval())

## hear we repeat X and Y  batch_size times
# twos = tf.ones([batch_size, 1 ,1], dtype = tf.int32)
# print(' twos: ', twos.shape)                
# twos_X = twos * X
# twos_Y = twos * Y
# print(' twos_exp * X', twos.shape, '*', X.shape, '= ',twos_X.shape)
# # print(twos_X[0,0].eval())
# print(' twos_exp * Y', twos.shape, '*', Y.shape, '= ',twos_Y.shape)
# print(twos_Y[0,0].eval())
twos_X = tf.tile(tf.expand_dims(X, axis = -1),[1,1,3])
twos_Y = tf.tile(tf.expand_dims(Y, axis = -1),[1,1,3])
print(' twos.shape X',twos_X.shape)
print(' twos_shape Y',twos_Y.shape)

thre_X = tf.tile(tf.expand_dims(twos_X, axis = -1),[1,1,1,32])
thre_Y = tf.tile(tf.expand_dims(twos_Y, axis = -1),[1,1,1,32])
print(' twos_X shape ', twos_X.shape, ' thre_X shape ',thre_X.shape)
print(' twos_Y shape ', twos_Y.shape, ' thre_Y shape ',thre_Y.shape)

# stack the X and Y grids 
grid = tf.to_float(tf.stack([thre_X,thre_Y], axis = -1))
print(grid.get_shape())

(8, 16) (8, 16)
 X : 
 [[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]]
 Y : 
 [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
 [3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
 [4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
 [5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]
 [6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6]
 [7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]]
 twos.shape X (8, 16, 3)
 twos_shape Y (8, 16, 3)
 twos_X shape  (8, 16, 3)  thre_X shape  (8, 16, 3, 32)
 twos_Y shape  (8, 16, 3)  thre_Y shape  (8, 16, 3, 32)
(8, 16, 3, 32, 2)


In [426]:
## Check equality of the two methods 

eq = tf.equal(grid, pos)
print( ' grid and pos equal --> ', tf.reduce_all(eq).eval())
# ass_eq = tf.equal(twos_X,rep_X)
# print(ass_eq.shape)
# equality = tf.reduce_all(ass_eq)
# print(equality.eval())        

 grid and pos equal -->  True


### Calculate mean and covariance matrices for gaussian distributions

In [427]:
# for ps in all_boxes:
ps =stacked_tensor
print(ps.get_shape())
width  = ps[:,:,5] - ps[:,:,3]
height = ps[:,:,4] - ps[:,:,2]
cx     = ps[:,:,3] + ( width  / 2.0)
cy     = ps[:,:,2] + ( height / 2.0)
means  = tf.stack((cx,cy),axis = -1)
covar  = tf.stack((width * 0.5 , height * 0.5), axis = -1)
# print(means.eval())
# print(covar.eval())

# print('width shape ',width.get_shape()) 
print('means shape ',means.get_shape())
print('covar shape ',covar.get_shape())


mns = means
cov = covar
# print(mns.eval())
print(' mns shape :', mns.shape, 'vcovar shape : ', cov.shape)
tfd = tf.contrib.distributions
mvn = tfd.MultivariateNormalDiag(
    loc  = mns,
    scale_diag = cov)

print(' mean shape ', mvn.mean().shape, 'stddev shape',mvn.stddev().shape)
print(' mvn.batch_shape ', mvn.batch_shape)
print(' mvn.event_shape ', mvn.event_shape)
# print(mvn.loc.eval())
print(' Linear OP shape', mvn.scale.shape, ' Linear Op batch shape ',mvn.scale.batch_shape)
print(' Linear op Range Dim ', mvn.scale.range_dimension)
print(' Linear op Domain Dim ', mvn.scale.domain_dimension)

(3, 32, 7)
means shape  (3, 32, 2)
covar shape  (3, 32, 2)
 mns shape : (3, 32, 2) vcovar shape :  (3, 32, 2)
 mean shape  (3, 32, 2) stddev shape (3, 32, 2)
 mvn.batch_shape  (3, 32)
 mvn.event_shape  (2,)
 Linear OP shape (3, 32, 2, 2)  Linear Op batch shape  (3, 32)
 Linear op Range Dim  2
 Linear op Domain Dim  2


In [457]:
inp = pos #grid
print(inp.get_shape())
# print(inp.eval())
# one_layer = tf.to_float(pos[0])
# print(one_layer[0].get_shape)
prob_pos = mvn.prob(inp)
print(' input (meshgrid) shape' , inp.get_shape())
print(' output (probabilities) shape', prob.get_shape())
# print(prob.eval())

inp = grid
print(inp.get_shape())
# print(inp.eval())
# one_layer = tf.to_float(pos[0])
# print(one_layer[0].get_shape)
prob_grid = mvn.prob(inp)
print(' input (meshgrid) shape' , inp.get_shape())
print(' output probabilities shape', prob.get_shape())
# print(prob.eval())
eq = tf.equal(grid, pos)
print( ' pos and grid probabalitiy matricies equal  -->', tf.reduce_all(eq).eval())

(8, 16, 3, 32, 2)
 input (meshgrid) shape (8, 16, 3, 32, 2)
 output (probabilities) shape (8, 16, 3, 32)
(8, 16, 3, 32, 2)
 input (meshgrid) shape (8, 16, 3, 32, 2)
 output probabilities shape (8, 16, 3, 32)
 pos and grid probabalitiy matricies equal  --> True


In [458]:
trans_grid= tf.transpose(prob_grid,[2,3,0,1])
print(trans_grid.shape)
trans_pos =  tf.transpose(prob_pos,[2,3,0,1])
print(trans_pos.shape)

for img in range(3):
    for idx in range(32):
        comapre = tf.equal(trans_grid[img,idx], trans_pos[img,idx])
        eq = tf.reduce_all(compare).eval()
        print( img , '/' ,idx, ' TRANSPOSED pos and grid probabalitiy matricies equal  -->', eq)
        if not eq :
            print(trans_grid[img,idx].eval())
            print(trans_pos[img,idx].eval())

# comapre = tf.equal(trans_grid[3], trans_pos[3])
# eq = tf.reduce_all(compare).eval()            
# print(eq)

(3, 32, 8, 16)
(3, 32, 8, 16)
0 / 0  TRANSPOSED pos and grid probabalitiy matricies equal  --> True
0 / 1  TRANSPOSED pos and grid probabalitiy matricies equal  --> True
0 / 2  TRANSPOSED pos and grid probabalitiy matricies equal  --> True
0 / 3  TRANSPOSED pos and grid probabalitiy matricies equal  --> True
0 / 4  TRANSPOSED pos and grid probabalitiy matricies equal  --> True
0 / 5  TRANSPOSED pos and grid probabalitiy matricies equal  --> True
0 / 6  TRANSPOSED pos and grid probabalitiy matricies equal  --> True
0 / 7  TRANSPOSED pos and grid probabalitiy matricies equal  --> True
0 / 8  TRANSPOSED pos and grid probabalitiy matricies equal  --> True
0 / 9  TRANSPOSED pos and grid probabalitiy matricies equal  --> True
0 / 10  TRANSPOSED pos and grid probabalitiy matricies equal  --> True
0 / 11  TRANSPOSED pos and grid probabalitiy matricies equal  --> True
0 / 12  TRANSPOSED pos and grid probabalitiy matricies equal  --> True
0 / 13  TRANSPOSED pos and grid probabalitiy matricies eq

### Build indicies to scatter probabiltiy distributions by class

In [459]:
class_inds = tf.to_int32(stacked_tensor[:,:,6])
batch_grid, roi_grid = tf.meshgrid( tf.range(batch_size, dtype=tf.int32), tf.range(num_rois, dtype=tf.int32), indexing = 'ij' )

print('class          shape: ', class_inds.shape)
print(class_inds.eval())
print('roi_grid shape',tf.shape(roi_grid).eval())
print(roi_grid.eval())
print('batch_grid     shape',tf.shape(batch_grid).eval())
print(batch_grid.eval())

scatter_classes = tf.stack([batch_grid, class_inds, roi_grid ],axis = -1)
print(scatter_classes.shape)
print(scatter_classes.eval())

class          shape:  (3, 32)
[[0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 3 3 3 3 3]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3]]
roi_grid shape [ 3 32]
[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]]
batch_grid     shape [ 3 32]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]]
(3, 32, 3)
[[[ 0  0  0]
  [ 0  0  1]
  [ 0  0  2]
  [ 0  0  3]
  [ 0  0  4]
  [ 0  0  5]
  [ 0  0  6]
  [ 0  0  7]
  [ 0  0  8]
  [ 0  0  9]
  [ 0  0 10]
  [ 0  0 11]
  [ 0  1 12]
  [ 0  1 13]
  [ 0  1 14]
  [ 0  1 

In [460]:
prob_scatt = tf.scatter_nd(scatter_classes, trans_pos, [batch_size, num_classes, num_rois, 8, 16])
print(prob_scatt.shape)
print(trans_grid.shape)

(3, 4, 32, 8, 16)
(3, 32, 8, 16)


In [413]:
prob_scatt[0,0,0].eval()
srt = 10
end = 17
img = 2
item = 21
clss = tf.to_int32(class_ids[img, item]).eval()
eq = tf.equal(prob_scatt[img,clss,item],trans_grid[img,item])
print( 'equal ', tf.reduce_all(eq).eval())

print(prob_scatt[img,clss,item].eval())
print(trans_grid[img,item].eval())

equal  True
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [461]:
for img in range(batch_size):
    for idx in range(rois_per_image):
        clss = tf.to_int32(class_ids[img, idx]).eval()
        
        compare  = tf.equal(prob_scatt[img,clss,idx],trans_grid[img,idx])
        eq =  tf.reduce_all(eq)
        print('img:', img,  'idx' , idx, 'class ', clss , 'equal ', eq.eval())
        if not eq.eval():
            print(prob_scatt[0,1,idx].eval())
            print(trans_grid[0,idx].eval())

img: 0 idx 0 class  0 equal  True
img: 0 idx 1 class  0 equal  True
img: 0 idx 2 class  0 equal  True
img: 0 idx 3 class  0 equal  True
img: 0 idx 4 class  0 equal  True
img: 0 idx 5 class  0 equal  True
img: 0 idx 6 class  0 equal  True
img: 0 idx 7 class  0 equal  True
img: 0 idx 8 class  0 equal  True
img: 0 idx 9 class  0 equal  True
img: 0 idx 10 class  0 equal  True
img: 0 idx 11 class  0 equal  True
img: 0 idx 12 class  1 equal  True
img: 0 idx 13 class  1 equal  True
img: 0 idx 14 class  1 equal  True
img: 0 idx 15 class  1 equal  True
img: 0 idx 16 class  1 equal  True
img: 0 idx 17 class  1 equal  True
img: 0 idx 18 class  2 equal  True
img: 0 idx 19 class  2 equal  True
img: 0 idx 20 class  2 equal  True
img: 0 idx 21 class  2 equal  True
img: 0 idx 22 class  2 equal  True
img: 0 idx 23 class  2 equal  True
img: 0 idx 24 class  2 equal  True
img: 0 idx 25 class  2 equal  True
img: 0 idx 26 class  2 equal  True
img: 0 idx 27 class  3 equal  True
img: 0 idx 28 class  3 equal  

In [462]:
prob_sum = tf.reduce_sum(prob_scatt, axis=2)
print(prob_sum.shape)

(3, 4, 8, 16)


In [473]:
for i in range(32):
    print(prob_scatt[0,0,i].eval())
          # prob_sum[0,2].eval()

[[4.1582e-25 4.3507e-25 4.5487e-25 4.7521e-25 4.9607e-25 5.1746e-25 5.3935e-25 5.6174e-25
  5.8462e-25 6.0796e-25 6.3174e-25 6.5596e-25 6.8058e-25 7.0558e-25 7.3095e-25 7.5665e-25]
 [9.3580e-25 9.7913e-25 1.0237e-24 1.0695e-24 1.1164e-24 1.1645e-24 1.2138e-24 1.2642e-24
  1.3157e-24 1.3682e-24 1.4217e-24 1.4762e-24 1.5316e-24 1.5879e-24 1.6450e-24 1.7028e-24]
 [2.0912e-24 2.1880e-24 2.2876e-24 2.3899e-24 2.4948e-24 2.6023e-24 2.7124e-24 2.8251e-24
  2.9401e-24 3.0574e-24 3.1771e-24 3.2988e-24 3.4227e-24 3.5484e-24 3.6760e-24 3.8052e-24]
 [4.6401e-24 4.8549e-24 5.0759e-24 5.3028e-24 5.5356e-24 5.7743e-24 6.0186e-24 6.2684e-24
  6.5237e-24 6.7841e-24 7.0495e-24 7.3197e-24 7.5945e-24 7.8735e-24 8.1566e-24 8.4433e-24]
 [1.0223e-23 1.0696e-23 1.1183e-23 1.1683e-23 1.2196e-23 1.2722e-23 1.3260e-23 1.3811e-23
  1.4373e-23 1.4947e-23 1.5532e-23 1.6127e-23 1.6732e-23 1.7347e-23 1.7971e-23 1.8602e-23]
 [2.2365e-23 2.3400e-23 2.4465e-23 2.5559e-23 2.6681e-23 2.7832e-23 2.9009e-23 3.0213e-23
  3.1

[[7.1001e-10 3.7379e-09 1.7653e-08 7.4786e-08 2.8421e-07 9.6887e-07 2.9628e-06 8.1277e-06
  2.0000e-05 4.4149e-05 8.7422e-05 1.5529e-04 2.4744e-04 3.5368e-04 4.5349e-04 5.2160e-04]
 [7.4502e-10 3.9223e-09 1.8524e-08 7.8474e-08 2.9822e-07 1.0167e-06 3.1090e-06 8.5285e-06
  2.0987e-05 4.6327e-05 9.1734e-05 1.6295e-04 2.5964e-04 3.7112e-04 4.7585e-04 5.4732e-04]
 [7.8095e-10 4.1115e-09 1.9417e-08 8.2259e-08 3.1261e-07 1.0657e-06 3.2589e-06 8.9398e-06
  2.1999e-05 4.8561e-05 9.6158e-05 1.7081e-04 2.7216e-04 3.8902e-04 4.9880e-04 5.7372e-04]
 [8.1777e-10 4.3053e-09 2.0332e-08 8.6137e-08 3.2735e-07 1.1159e-06 3.4125e-06 9.3613e-06
  2.3036e-05 5.0850e-05 1.0069e-04 1.7886e-04 2.8499e-04 4.0736e-04 5.2232e-04 6.0077e-04]
 [8.5544e-10 4.5036e-09 2.1269e-08 9.0105e-08 3.4242e-07 1.1673e-06 3.5697e-06 9.7925e-06
  2.4097e-05 5.3192e-05 1.0533e-04 1.8710e-04 2.9812e-04 4.2612e-04 5.4638e-04 6.2844e-04]
 [8.9391e-10 4.7061e-09 2.2226e-08 9.4157e-08 3.5782e-07 1.2198e-06 3.7303e-06 1.0233e-05
  2.5

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 

# Manipulate 3D Tensors

###  Create class score tensor
 1- Class_Score tensor:   batch_size x  num_dectections x num_classes
 
 2- Output_ROIs :         batch_size x  num_detections  x 4   
 
 3- mrcnn_bboxes:         batch_size x  num_detections  x num_clases x 4

In [3]:
mrcnn_class_1 = tf.Variable(tf.random_uniform([batch_size, num_rois-2, num_classes], minval=0, maxval=1  , dtype=tf.float32, seed=1234), name = 'var')
output_rois_1 = tf.Variable(tf.random_uniform([batch_size, num_rois-2, 4], minval=0, maxval=128, dtype=tf.float32, seed=1234), name = 'var')
mrcnn_bboxes  = tf.Variable(tf.random_uniform([batch_size, num_rois, num_classes, 4], minval=0, maxval=128, dtype=tf.float32, seed=1234), name = 'var')

mrcnn_class_filler = tf.zeros([batch_size, 2, num_classes], dtype = tf.float32)
output_rois_filler = tf.zeros([batch_size, 2, 4], dtype = tf.float32)

one   = tf.ones([num_rois, 4])
test_bboxes = tf.stack([one, one+1, one+2, one+3], axis = 0) 
init = tf.global_variables_initializer().run()

mrcnn_class = tf.concat([mrcnn_class_1, mrcnn_class_filler],axis = 1)
output_rois = tf.concat([output_rois_1, output_rois_filler],axis = 1)

print(' mrcnn_class shape: ', mrcnn_class.shape)
print(' output_rois shape: ', output_rois.shape)
print('  Scores .shape', mrcnn_class.get_shape() )
print(mrcnn_class.eval())
# print(output_rois.eval())

 mrcnn_class shape:  (3, 32, 4)
 output_rois shape:  (3, 32, 4)
  Scores .shape (3, 32, 4)
[[[0.8483 0.3236 0.3067 0.0697]
  [0.9139 0.1705 0.2834 0.3563]
  [0.5416 0.0753 0.0745 0.866 ]
  [0.5591 0.0093 0.067  0.4977]
  [0.2989 0.8094 0.6585 0.4815]
  [0.1517 0.591  0.3344 0.1552]
  [0.2455 0.1395 0.9869 0.9031]
  [0.1522 0.023  0.7943 0.6682]
  [0.7623 0.9574 0.8136 0.2078]
  [0.8659 0.1014 0.2265 0.0317]
  [0.1861 0.4301 0.1205 0.2435]
  [0.3574 0.1943 0.192  0.2904]
  [0.8358 0.9281 0.8308 0.0583]
  [0.3079 0.9126 0.3954 0.7867]
  [0.3893 0.0964 0.6395 0.6672]
  [0.7801 0.5502 0.5758 0.4603]
  [0.5407 0.5492 0.2173 0.6376]
  [0.482  0.0279 0.954  0.9278]
  [0.7984 0.7105 0.8229 0.7032]
  [0.4165 0.0972 0.9167 0.4561]
  [0.1692 0.3054 0.6161 0.5193]
  [0.6092 0.147  0.124  0.0996]
  [0.3477 0.3669 0.3179 0.4279]
  [0.9821 0.3264 0.8513 0.0144]
  [0.7459 0.5724 0.9563 0.6972]
  [0.1117 0.7005 0.66   0.7265]
  [0.3661 0.5429 0.7805 0.6744]
  [0.2175 0.2784 0.5751 0.1126]
  [0.9919 0.7

###  Build indicies to gather bounding boxes from bboxes_4d corrsponding to predicted class
#### Only used if we want to use mrcnn_bboxes (batch_size, num_rois, num_classes, 4)

batch_size x nuum_detections x 4

In [52]:
# gather_boxes = tf.stack([batch_grid, roi_grid, pred_classes, ], axis = -1)

# print('-- gather_boxes  ----')
# print('gather_boxes inds', type(gather_boxes), 'shape',tf.shape(gather_boxes).eval())
# print(gather_boxes.eval())

# mrcnn_bboxes_selected = tf.gather_nd(mrcnn_bboxes, gather_boxes)
# print(' padding required for output_rois : ', mrcnn_bboxes_selected.get_shape())
# print(mrcnn_bboxes_selected[0].eval())

print(' output_rois shape ', output_rois.get_shape())
print(pred_classes[0].eval())
print(output_rois[0].eval())



 output_rois shape  (3, 32, 4)
[0 0 3 0 1 1 2 2 1 0 1 0 1 1 3 0 3 2 2 2 2 0 3 0 2 3 2 2 0 0 0 0]
[[108.5833  41.4171  39.2576   8.9216]
 [116.9736  21.8213  36.2715  45.6026]
 [ 69.3192   9.6329   9.5347 110.8421]
 [ 71.5596   1.1958   8.5777  63.7113]
 [ 38.2599 103.6086  84.2878  61.6374]
 [ 19.4185  75.6502  42.8045  19.8604]
 [ 31.423   17.8502 126.3289 115.5957]
 [ 19.4829   2.9421 101.6725  85.5338]
 [ 97.5796 122.5435 104.1375  26.6011]
 [110.8402  12.9817  28.9916   4.0611]
 [ 23.823   55.0586  15.4241  31.1665]
 [ 45.7513  24.8754  24.5708  37.1656]
 [106.9807 118.7934 106.3466   7.4645]
 [ 39.4174 116.8137  50.6049 100.7003]
 [ 49.8244  12.3429  81.8621  85.3981]
 [ 99.8521  70.4273  73.7088  58.9241]
 [ 69.2067  70.3032  27.8131  81.6126]
 [ 61.6912   3.5655 122.1108 118.7579]
 [102.1897  90.9454 105.3337  90.0073]
 [ 53.3086  12.4369 117.3345  58.387 ]
 [ 21.658   39.0892  78.8661  66.4719]
 [ 77.9712  18.8217  15.8669  12.7541]
 [ 44.5081  46.9678  40.6923  54.7697]
 [125.

###  Generate pred_array: tensor [ seq_id,  max_score,  bbox coordinates, class_id]
####  1 -  Get max prediction score and  corresponding class for each ROI
####  2 -  Concat sequence id, max_score, bbox coordinates and class id 

In [53]:
pred_classes     = tf.argmax(mrcnn_class,axis=-1, output_type = tf.int32)
pred_classes_exp = tf.to_float(tf.expand_dims(pred_classes,axis=-1))
print('  pred_classes (classes with highest scores):', pred_classes.get_shape())
print(pred_classes.eval())
print('  predclasses expanded shape:', pred_classes_exp.get_shape())

 
pred_scores  = tf.reduce_max(mrcnn_class,axis=-1, keepdims=True)
print('  pred_scores shape', pred_scores.shape)
# print(pred_scores.eval())

pred_array = tf.concat([ bbox_idx, pred_scores , output_rois, pred_classes_exp], axis=-1)

print('  resulting tensor : pred_array',  type(pred_array), pred_array.shape)
print(pred_array[0].eval())
print('------')
print(pred_array[1].eval())
print('------')
print(pred_array[2].eval())

  pred_classes (classes with highest scores): (3, 32)
[[0 0 3 0 1 1 2 2 1 0 1 0 1 1 3 0 3 2 2 2 2 0 3 0 2 3 2 2 0 0 0 0]
 [0 3 1 1 3 3 0 3 2 0 3 3 3 3 3 3 1 3 0 2 3 1 0 0 1 2 0 0 3 3 0 0]
 [0 0 1 3 1 3 3 0 3 0 2 3 1 0 0 1 1 1 3 3 2 3 0 3 3 2 2 0 3 2 0 0]]
  predclasses expanded shape: (3, 32, 1)
  pred_scores shape (3, 32, 1)
  resulting tensor : pred_array <class 'tensorflow.python.framework.ops.Tensor'> (3, 32, 7)
[[  0.       0.8483 108.5833  41.4171  39.2576   8.9216   0.    ]
 [  1.       0.9139 116.9736  21.8213  36.2715  45.6026   0.    ]
 [  2.       0.866   69.3192   9.6329   9.5347 110.8421   3.    ]
 [  3.       0.5591  71.5596   1.1958   8.5777  63.7113   0.    ]
 [  4.       0.8094  38.2599 103.6086  84.2878  61.6374   1.    ]
 [  5.       0.591   19.4185  75.6502  42.8045  19.8604   1.    ]
 [  6.       0.9869  31.423   17.8502 126.3289 115.5957   2.    ]
 [  7.       0.7943  19.4829   2.9421 101.6725  85.5338   2.    ]
 [  8.       0.9574  97.5796 122.5435 104.1375  26.6


####  2 -  Generate meshgrid of image_id (batch grid) and bbox_ids  (roi_grid)
####  3 -  Generate scatter indices to scatter bbox information by image_id and class_id 
The the highest score for each detection (highest score amongst classes) and the class corresponding to the highest score 

In [54]:
batch_grid, roi_grid = tf.meshgrid( tf.range(batch_size, dtype=tf.int32), tf.range(num_rois, dtype=tf.int32), indexing = 'ij' )

print('pred_class shape: ', pred_classes.shape)
print(pred_classes.eval())
print('roi_grid shape',tf.shape(roi_grid).eval())
print(roi_grid.eval())
print('batch_grid     shape',tf.shape(batch_grid).eval())
print(batch_grid.eval())

bbox_idx        = tf.to_float(tf.expand_dims(roi_grid, axis = -1))
scatter_classes = tf.stack([batch_grid , pred_classes, roi_grid],axis = -1)
print('-- scatter to classes ----')
print('scatter_classes', type(scatter_classes), 'shape',tf.shape(scatter_classes).eval())
print(scatter_classes.eval())

pred_class shape:  (3, 32)
[[0 0 3 0 1 1 2 2 1 0 1 0 1 1 3 0 3 2 2 2 2 0 3 0 2 3 2 2 0 0 0 0]
 [0 3 1 1 3 3 0 3 2 0 3 3 3 3 3 3 1 3 0 2 3 1 0 0 1 2 0 0 3 3 0 0]
 [0 0 1 3 1 3 3 0 3 0 2 3 1 0 0 1 1 1 3 3 2 3 0 3 3 2 2 0 3 2 0 0]]
roi_grid shape [ 3 32]
[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]]
batch_grid     shape [ 3 32]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]]
-- scatter to classes ----
scatter_classes <class 'tensorflow.python.framework.ops.Tensor'> shape [ 3 32  3]
[[[ 0  0  0]
  [ 0  0  1]
  [ 0  3  2]
  [ 0  0  3]
  [ 0  1  4]
  [ 0  1  5]
  [ 0  2  6]
  [ 0  2  7]
  [ 0 

###  Split Box/ Class info for each Image dim(0) by class and place into new tensor using tf.scatter_nd

    Output is converted to NUMN_BATCHES x NUM_CLASSES x NUM_ROIS x columns(7)

In [55]:
pred_scatt = tf.scatter_nd(scatter_classes, pred_array, [batch_size, num_classes, num_rois,7])
print('pred_scatter shape is ', pred_scatt.get_shape())
np.set_printoptions(linewidth=100, precision=4)
print('------------')
print(pred_scatt[0].eval())
print('------------')
np.set_printoptions(linewidth=100, precision=4)
print(pred_scatt[1].eval())
print('------------')

print(pred_scatt[2].eval())

# equality = tf.equal(pred_scatt[0,0,0], pred_scatt2[0,0,0])
# print(tf.reduce_all(equality).eval())

pred_scatter shape is  (3, 4, 32, 7)
------------
[[[  0.       0.8483 108.5833  41.4171  39.2576   8.9216   0.    ]
  [  1.       0.9139 116.9736  21.8213  36.2715  45.6026   0.    ]
  [  0.       0.       0.       0.       0.       0.       0.    ]
  [  3.       0.5591  71.5596   1.1958   8.5777  63.7113   0.    ]
  [  0.       0.       0.       0.       0.       0.       0.    ]
  [  0.       0.       0.       0.       0.       0.       0.    ]
  [  0.       0.       0.       0.       0.       0.       0.    ]
  [  0.       0.       0.       0.       0.       0.       0.    ]
  [  0.       0.       0.       0.       0.       0.       0.    ]
  [  9.       0.8659 110.8402  12.9817  28.9916   4.0611   0.    ]
  [  0.       0.       0.       0.       0.       0.       0.    ]
  [ 11.       0.3574  45.7513  24.8754  24.5708  37.1656   0.    ]
  [  0.       0.       0.       0.       0.       0.       0.    ]
  [  0.       0.       0.       0.       0.       0.       0.    ]
  [  0.     

###  Sort pred_scatter by score 
    Sort each slice based on column  1 (score) using top_k
    Generate calss_grid, batch_grid and roi-grid based using mesh_grid 
    Stack results to generate gather indices 
    Generate final result - pred_tensor - using tf.gather_nd()         

In [59]:
sort_vals, sort_inds = tf.nn.top_k(pred_scatt[:,:,:,1], k=pred_scatt.shape[2])

# print(tf.shape(sort_vals).eval())
# print(sort_vals.eval())
print()
print(tf.shape(sort_inds).eval())
print(sort_inds.eval())

class_grid, batch_grid, dtct_grid = tf.meshgrid(tf.range(num_classes),tf.range(batch_size), tf.range(num_rois))
gather_inds = tf.stack([batch_grid , class_grid, sort_inds],axis = -1)

# print('class_grid', type(class_grid), 'shape',tf.shape(class_grid).eval())
# print(class_grid.eval())

# print('batch_grid', type(batch_grid), 'shape',tf.shape(batch_grid).eval())
# print(batch_grid.eval())

# print('class_grid', type(dtct_grid), 'shape',tf.shape(dtct_grid).eval())
# print(dtct_grid.eval())

print('-- stack results ----')
print('gather_inds', type(gather_inds), 'shape',tf.shape(gather_inds).eval())
print(gather_inds.eval())


pred_tensor = tf.gather_nd(pred_scatt, gather_inds)

print()
print('-- gather_nd results (A-boxes sorted by score ----')
print(' srtd_boxes shape : ', tf.shape(pred_tensor).eval())
print(pred_tensor.eval())


[ 3  4 32]
[[[28 23 29  1  9  0 15 21  3 11  2  4  5  6  7  8 10 12 13 14 16 17 18 19 20 22 24 25 26 27 30
   31]
  [ 8 12 13  4  5 10  0  1  2  3  6  7  9 11 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30
   31]
  [ 6 24 17 19 18  7 26 20 27  0  1  2  3  4  5  8  9 10 11 12 13 14 15 16 21 22 23 25 28 29 30
   31]
  [ 2 25 14 16 22  0  1  3  4  5  6  7  8  9 10 11 12 13 15 17 18 19 20 21 23 24 26 27 28 29 30
   31]]

 [[26  9  0 22 23 18 27  6  1  2  3  4  5  7  8 10 11 12 13 14 15 16 17 19 20 21 24 25 28 29 30
   31]
  [24 16  3  2 21  0  1  4  5  6  7  8  9 10 11 12 13 14 15 17 18 19 20 22 23 25 26 27 28 29 30
   31]
  [25  8 19  0  1  2  3  4  5  6  7  9 10 11 12 13 14 15 16 17 18 20 21 22 23 24 26 27 28 29 30
   31]
  [ 5 13 17 28  4 20 15 14 29  1  7 11 12 10  0  2  3  6  8  9 16 18 19 21 22 23 24 25 26 27 30
   31]]

 [[ 1  7  9 13  0 22 14 27  2  3  4  5  6  8 10 11 12 15 16 17 18 19 20 21 23 24 25 26 28 29 30
   31]
  [15 12 16  2 17  4  0  1  3  5  6  7  8  9 10 11 13 14 

###  Display for visual check

In [60]:
np.set_printoptions(linewidth=100, precision=4)
print('scatter shape is ', pred_scatt.get_shape(), pred_scatt)
img  = 1
# print(pred_scatt[img,0].eval())
print(pred_tensor[img,0].eval())

print('------------')
# print(pred_scatt[img, 1].eval())
print(pred_tensor[img,1].eval())

print('------------')
# print(pred_scatt[img,2].eval())
print(pred_tensor[img, 2].eval())

print('------------')
# print(pred_scatt[img,3].eval())
print(pred_tensor[img, 3].eval())


scatter shape is  (3, 4, 32, 7) Tensor("ScatterNd_5:0", shape=(3, 4, 32, 7), dtype=float32)
[[ 26.       0.988  126.4669  29.1784  95.5444 114.4807   0.    ]
 [  9.       0.9796 125.3944  48.341   85.1252  24.5082   0.    ]
 [  0.       0.9445 120.8985  56.4598  65.6325  11.3239   0.    ]
 [ 22.       0.8294 106.1611   3.1695  62.8515  32.9459   0.    ]
 [ 23.       0.792  101.3707  95.1844   1.9615  96.4102   0.    ]
 [ 18.       0.725   92.8012  23.3539   9.767   44.9132   0.    ]
 [ 27.       0.6424  82.2321  41.9108  58.0751  68.8583   0.    ]
 [  6.       0.6395  81.8618  17.9545  60.2488  71.9641   0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       0.       0

In [85]:
# tf.count_nonzero(pred_tensor[0,0]).eval()

In [213]:
a00 = pred_tensor
print(a00[:,:,:,0].get_shape())

a00_cnt = tf.count_nonzero(a00[:,:,:,0],axis = -1)
print(a00_cnt.get_shape())
a00_cnt.eval()

(3, 4, 28)
(3, 4)


array([[16,  3,  5,  3],
       [16,  7,  2,  2],
       [14,  5,  5,  3]], dtype=int64)

###  Sort each slice based on class id (obsolete)

#### Now we split based on class id first then sort based on score - no need to sort on class beforehand

In [11]:
# sort_vals, sort_inds = tf.nn.top_k(-pred_array[:,:,6], k=pred_array.shape[1])

# print(tf.shape(sort_vals).eval())
# print(sort_vals.eval())
# print()
# print(tf.shape(sort_inds).eval())
# print(sort_inds.eval())

###  Scatter to class layers - Loop / Dynamic Partition method  (Obsolete)
 
We are scattering using the scatter_nd_update method

In [15]:
print('shape of srtd_boxes ', srtd_boxes.shape)
partition_inds = tf.to_int32(srtd_boxes[...,6])
print(partition_inds.eval())


batch_stack = []
for i in range(batch_size):
    
    splitted = tf.dynamic_partition(srtd_boxes[i], partition_inds[i], num_classes)

    for idx, split in enumerate(splitted):
#         print(' item ', idx,' shape ',split.shape, 'shape:', tf.shape(split).eval(), 'rank: ', tf.rank(split).eval())
        padding = num_rois - tf.shape(split)[0]
        split = tf.pad(split,[[0,padding],[0,0]]) 
#         print(' num of padding rows tt add ', padding.eval())        
#         print(' shape after padding : ', tf.shape(split).eval())
#         print(split.eval())
        splitted[idx] = split
    stacked = tf.stack(splitted, axis = 0)
    batch_stack.append(stacked)

batch_tensor = tf.stack(batch_stack, axis = 0)
print(' batch _tensor shape is ', tf.shape(batch_tensor).eval())
print(batch_tensor[0].eval())    


shape of srtd_boxes  (3, 28, ?)
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 2 2 3 3 3]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 2 2 2 2 2 2 2 3 3 3]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 2 3 3 3 3 3 3 3 3 3 3 3]]
 batch _tensor shape is  [ 3  4 28  7]
[[[  0.       0.8483 108.5833  41.4171  39.2576   8.9216   0.    ]
  [  1.       0.9139 116.9736  21.8213  36.2715  45.6026   0.    ]
  [  3.       0.5591  71.5596   1.1958   8.5777  63.7113   0.    ]
  [  9.       0.8659 110.8402  12.9817  28.9916   4.0611   0.    ]
  [ 11.       0.3574  45.7513  24.8754  24.5708  37.1656   0.    ]
  [ 15.       0.7801  99.8521  70.4273  73.7088  58.9241   0.    ]
  [ 17.       0.      61.6912   3.5655 122.1108 118.7579   0.    ]
  [ 18.       0.     102.1897  90.9454 105.3337  90.0073   0.    ]
  [ 19.       0.      53.3086  12.4369 117.3345  58.387    0.    ]
  [ 20.       0.      21.658   39.0892  78.8661  66.4719   0.    ]
  [ 21.       0.      77.9712  18.8217  15.8669  12.7541   0.    ]
  [ 22.       

## Experimental Code

###  Experiment for sort ordering 

In [462]:
indices = tf.constant([
            [[0,2],[0,1],[0,0]],
            [[1,2],[1,1],[1,0]],
            [[2,0],[2,1],[2,1]],
          ])
params = tf.constant( [
          [['0-00', '0-01', '0-02', '0-03'], 
           ['0-10', '0-11', '0-12', '0-13'],
           ['0-20', '0-21', '0-22', '0-23'],
           ['0-30', '0-31', '0-32', '0-33']],
    
          [['1-00', '1-01', '1-02', '1-03'], 
           ['1-10', '1-11', '1-12', '1-13'],
           ['1-20', '1-21', '1-22', '1-23'],
           ['1-30', '1-31', '1-32', '1-33']],
    
          [['2-00', '2-01', '2-02', '2-03'], 
           ['2-10', '2-11', '2-12', '2-13'],
           ['2-20', '2-21', '2-22', '2-23'],
           ['2-30', '2-31', '2-32', '2-33']]    
         ])
print(params.shape, '   ', indices.shape)
res = tf.gather_nd(params, indices)
res.eval()



(3, 4, 4)     (3, 3, 2)


array([[[b'0-20', b'0-21', b'0-22', b'0-23'],
        [b'0-10', b'0-11', b'0-12', b'0-13'],
        [b'0-00', b'0-01', b'0-02', b'0-03']],

       [[b'1-20', b'1-21', b'1-22', b'1-23'],
        [b'1-10', b'1-11', b'1-12', b'1-13'],
        [b'1-00', b'1-01', b'1-02', b'1-03']],

       [[b'2-00', b'2-01', b'2-02', b'2-03'],
        [b'2-10', b'2-11', b'2-12', b'2-13'],
        [b'2-10', b'2-11', b'2-12', b'2-13']]], dtype=object)

###  Experiment with GATHER() for splitting into classes

In [84]:
# indices = tf.constant([
#             [0,2,2],[0,1,1],[0,0,0],
#             [1,2,2],[1,1,1],[1,0,0],
#             [2,0,0],[2,1,1],[2,1,1],
#            ])
# indices = tf.constant([ [0,2],[0,1],[0,0],[1,2],[1,1],[1,0],[1,2],[1,3], [2,3]] ) # 9 x 2
indices = tf.constant([
           [
            [[0,2],[0,1],[0,0],[1,2],[2,3]],
            [[1,1],[1,0],[1,2],[1,3],[1,1]]
           ]
           ])
params = tf.constant( [
          [['0-00', '0-01', '0-02', '0-03', '0-04', '0-05', '0-06'], 
           ['0-10', '0-11', '0-12', '0-13', '0-14', '0-15', '0-16'],
           ['0-20', '0-21', '0-22', '0-23', '0-24', '0-25', '0-26'],
           ['0-30', '0-31', '0-32', '0-33', '0-34', '0-35', '0-36'],
           ['0-40', '0-41', '0-42', '0-43', '0-44', '0-45', '0-46']],
                                                          
          [['1-00', '1-01', '1-02', '1-03', '1-04', '1-05', '1-06'], 
           ['1-10', '1-11', '1-12', '1-13', '1-14', '1-15', '1-16'],
           ['1-20', '1-21', '1-22', '1-23', '1-24', '1-25', '1-26'],
           ['1-30', '1-31', '1-32', '1-33', '1-34', '1-35', '1-36'],
           ['1-40', '1-41', '1-42', '1-43', '1-44', '1-45', '1-46']],
                                                          
          [['2-00', '2-01', '2-02', '2-03', '2-04', '2-05', '2-06'], 
           ['2-10', '2-11', '2-12', '2-13', '2-14', '2-15', '2-16'],
           ['2-20', '2-21', '2-22', '2-23', '2-24', '2-25', '2-26'],
           ['2-30', '2-31', '2-32', '2-33', '2-34', '2-35', '2-36'],
           ['2-40', '2-41', '2-42', '2-43', '2-44', '2-45', '2-46']]    
         ])
print(' params sahape: ', params.shape, ' parms[:axis]', params.shape[:0], params.shape[:1], params.shape[:2], params.shape[:3])
print(' params sahape: ', params.shape, ' parms[:axis]', params.shape[0:], params.shape[1:], params.shape[2:], params.shape[3:])
print(' indices.shape  ', indices.shape)
res = tf.gather_nd(params, indices)
print('result shape   ', res.get_shape())
res.eval()

 params sahape:  (3, 5, 7)  parms[:axis] () (3,) (3, 5) (3, 5, 7)
 params sahape:  (3, 5, 7)  parms[:axis] (3, 5, 7) (5, 7) (7,) ()
 indices.shape   (1, 2, 5, 2)
result shape    (1, 2, 5, 7)


array([[[[b'0-20', b'0-21', b'0-22', b'0-23', b'0-24', b'0-25', b'0-26'],
         [b'0-10', b'0-11', b'0-12', b'0-13', b'0-14', b'0-15', b'0-16'],
         [b'0-00', b'0-01', b'0-02', b'0-03', b'0-04', b'0-05', b'0-06'],
         [b'1-20', b'1-21', b'1-22', b'1-23', b'1-24', b'1-25', b'1-26'],
         [b'2-30', b'2-31', b'2-32', b'2-33', b'2-34', b'2-35', b'2-36']],

        [[b'1-10', b'1-11', b'1-12', b'1-13', b'1-14', b'1-15', b'1-16'],
         [b'1-00', b'1-01', b'1-02', b'1-03', b'1-04', b'1-05', b'1-06'],
         [b'1-20', b'1-21', b'1-22', b'1-23', b'1-24', b'1-25', b'1-26'],
         [b'1-30', b'1-31', b'1-32', b'1-33', b'1-34', b'1-35', b'1-36'],
         [b'1-10', b'1-11', b'1-12', b'1-13', b'1-14', b'1-15', b'1-16']]]], dtype=object)

###  Experiment with SCATTER() for splitting into classes

In [83]:
# del indices, updates, shape

indices =  tf.constant([
                         #|
                         #| 
                        [[0,0,0], [0,3,0], [0,2,0], [0,1,0],[0,1,1],[0,1,2],[0,2,1],[0,3,2] ],
                        [[1,0,0], [1,3,0], [1,2,0], [1,1,0],[1,1,1],[1,1,2],[1,2,1],[1,3,2] ],
                        [[2,0,0], [2,3,0], [2,2,0], [2,1,0],[2,1,1],[2,1,2],[2,2,1],[2,3,2] ]
])
                            #<--- each one corrsponds to an entry in  the updates array [row , position in resulting array]
    
#                        [[1,1], [3,1], [2,1], [1,1],[1,1],[1,2],[2,1],[3,2] ],
#                        [[0,0], [3,0], [2,0], [1,0],[1,1],[1,2],[2,1],[3,2] ]
    
###----------------------------------------------------
### Our bounding box array
###---------------------------------------------------
updates = tf.constant( [
          [[1000, 1001, 1002, 1003, 1004, 1005, 1006], 
           [1010, 1011, 1012, 1013, 1014, 1015, 1016],
           [1020, 1021, 1022, 1023, 1024, 1025, 1026],
           [1030, 1031, 1032, 1033, 1034, 1035, 1036],
           [1040, 1041, 1042, 1043, 1044, 1045, 1046],
           [1050, 1051, 1052, 1053, 1054, 1055, 1056],
           [1060, 1061, 1062, 1063, 1064, 1065, 1066],
           [1070, 1071, 1072, 1073, 1074, 1075, 1076] ],
          
          [[2000, 2001, 2002, 2003, 2004, 2005, 2006], 
           [2010, 2011, 2012, 2013, 2014, 2015, 2016],
           [2020, 2021, 2022, 2023, 2024, 2025, 2026],
           [2030, 2031, 2032, 2033, 2034, 2035, 2036],
           [2040, 2041, 2042, 2043, 2044, 2045, 2046],
           [2050, 2051, 2052, 2053, 2054, 2055, 2056],
           [2060, 2061, 2062, 2063, 2064, 2065, 2066],
           [2070, 2071, 2072, 2073, 2074, 2075, 2076]] , 

          [[3000, 3001, 3002, 3003, 3004, 3005, 3006], 
           [3010, 3011, 3012, 3013, 3014, 3015, 3016],
           [3020, 3021, 3022, 3023, 3024, 3025, 3026],
           [3030, 3031, 3032, 3033, 3034, 3035, 3036],
           [3040, 3041, 3042, 3043, 3044, 3045, 3046],
           [3050, 3051, 3052, 3053, 3054, 3055, 3056],
           [3060, 3061, 3062, 3063, 3064, 3065, 3066],
           [3070, 3071, 3072, 3073, 3074, 3075, 3076] ]    
         ])                       

zeros = tf.zeros([3,4,8,7], dtype=tf.int32)
ref   = tf.Variable(zeros)
scatt1= tf.Variable(zeros)
shape = tf.constant([3,4, 8, 7])
 
# init = tf.global_variables_initializer().run()
# print('indices shape ', indices.shape,  'indices_shape [-1]', indices.shape[-1],  'indices_shape [:-1]', indices.shape[:-1])
# print('updates.shape ', updates.shape)
# print('ref     shape ', ref.get_shape(), ref)
# print('scatt1  shape ', scatt1.get_shape(), scatt)

scatt1 = tf.scatter_nd(indices, updates, shape)
# print('scatt1  shape ', scatt1.get_shape(), scatt)
# print(scatt1[0].eval())
# print('------------')
# print(scatt1[1].eval())
# print('------------')
# print(scatt1[2].eval())

# scatt = tf.scatter_nd_update(ref, indices, updates)
# print('scatter shape is ', scatt.get_shape(), scatt)
# print(scatt.eval())

print()


indices shape  (3, 8, 3) indices_shape [-1] 3 indices_shape [:-1] (3, 8)
updates.shape  (3, 8, 7)
ref     shape  (3, 4, 8, 7) <tf.Variable 'Variable_113:0' shape=(3, 4, 8, 7) dtype=int32_ref>
scatt1  shape  (3, 4, 8, 7) <tf.Variable 'Variable_102:0' shape=(3, 4, 8, 7) dtype=int32_ref>
scatt1  shape  (3, 4, 8, 7) <tf.Variable 'Variable_102:0' shape=(3, 4, 8, 7) dtype=int32_ref>
[[[1000 1001 1002 1003 1004 1005 1006]
  [   0    0    0    0    0    0    0]
  [   0    0    0    0    0    0    0]
  [   0    0    0    0    0    0    0]
  [   0    0    0    0    0    0    0]
  [   0    0    0    0    0    0    0]
  [   0    0    0    0    0    0    0]
  [   0    0    0    0    0    0    0]]

 [[1030 1031 1032 1033 1034 1035 1036]
  [1040 1041 1042 1043 1044 1045 1046]
  [1050 1051 1052 1053 1054 1055 1056]
  [   0    0    0    0    0    0    0]
  [   0    0    0    0    0    0    0]
  [   0    0    0    0    0    0    0]
  [   0    0    0    0    0    0    0]
  [   0    0    0    0    0    0 

In [49]:
indices = tf.constant([
                        [[0,0], [3,0], [2,0], [1,0]]
#                     ,[[1,3], [1,3], [1,2], [1,0]]
                      ])
updates = tf.constant([
                       [[5, 5, 5, 5, 0], 
                        [6, 6, 6, 6, 3],
                        [7, 7, 7, 7, 2], 
                        [8, 8, 8, 8, 1]]
                       
#                      , [[5, 5, 5, 3], 
#                         [6, 6, 6, 3],
#                         [7, 7, 7, 2], 
#                         [8, 8, 8, 0]]
                       ])
shape = tf.constant([4, 4, 5])
print(indices.shape, 'inputs shape', updates.shape, shape.shape)
scatter = tf.scatter_nd(indices, updates, shape)
print('scatter shape is ', scatter.get_shape())
print(scatter.eval())

(1, 4, 2) inputs shape (1, 4, 5) (3,)
scatter shape is  (4, 4, 5)
[[[5 5 5 5 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[8 8 8 8 1]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[7 7 7 7 2]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[6 6 6 6 3]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]


In [70]:
num_detections = 8
# batch_size is  3
gt_class_ids = tf.constant([[1,2,3,0,0,0,0,0],[3,3,3,0,0,0,0,0],[1,2,2,0,0,2,0,0]])
gt_bboxes    = tf.random_uniform([batch_size,3,4], maxval = 127, dtype=tf.int32)


gt_classes_exp = tf.to_float(tf.expand_dims(gt_class_ids ,axis=-1))
print('    gt_classes_exp: ' ,gt_classes_exp.get_shape())
print(gt_classes_exp.eval())


zeros        = tf.zeros([batch_size,5,4], dtype = tf.int32)
gt_bboxes    = tf.concat([gt_bboxes, zeros], axis = 1)
gt_bboxes    = tf.to_float(gt_bboxes)
print('\n    gt_bboxes: ' ,gt_bboxes.get_shape())
print(gt_bboxes.eval())

mask = tf.greater(gt_class_ids,0)
print(mask.eval())

gt_scores     = tf.where(mask, tf.ones_like(gt_class_ids), tf.zeros_like(gt_class_ids))
gt_scores_exp = tf.to_float(tf.expand_dims(gt_scores, axis=-1))
print('\n    gt_scores ', gt_scores_exp.get_shape())
print(gt_scores_exp.eval())

batch_grid, bbox_grid = tf.meshgrid( tf.range(batch_size    , dtype=tf.int32), 
                                    tf.range(num_detections , dtype=tf.int32), indexing = 'ij' )

print('\n    bbox_grid   ', type(bbox_grid)  , 'shape', bbox_grid.get_shape())
print(bbox_grid.eval())
print('\n    batch_grid ', type(batch_grid), 'shape', batch_grid.get_shape())
print(batch_grid.eval())
 
bbox_idx_zeros  = tf.zeros_like(bbox_grid)
bbox_idx        = tf.where(mask, bbox_grid , bbox_idx_zeros)
bbox_idx        = tf.to_float(tf.expand_dims(bbox_idx, axis = -1))    
print('    bbox_idx', type(bbox_idx), 'shape', bbox_idx.get_shape())
print(bbox_idx.eval())

    gt_classes_exp:  (3, 8, 1)
[[[1.]
  [2.]
  [3.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]]

 [[3.]
  [3.]
  [3.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]]

 [[1.]
  [2.]
  [2.]
  [0.]
  [0.]
  [2.]
  [0.]
  [0.]]]

    gt_bboxes:  (3, 8, 4)
[[[ 90. 123.  82.  90.]
  [ 98.  49.  65. 110.]
  [ 93.  21.  66.  21.]
  [  0.   0.   0.   0.]
  [  0.   0.   0.   0.]
  [  0.   0.   0.   0.]
  [  0.   0.   0.   0.]
  [  0.   0.   0.   0.]]

 [[ 48.  13. 109. 125.]
  [108.  25.  33.  18.]
  [120.  62.  35. 102.]
  [  0.   0.   0.   0.]
  [  0.   0.   0.   0.]
  [  0.   0.   0.   0.]
  [  0.   0.   0.   0.]
  [  0.   0.   0.   0.]]

 [[124.  52.  83.  95.]
  [117.  21. 109.  57.]
  [114.  71.  59.  73.]
  [  0.   0.   0.   0.]
  [  0.   0.   0.   0.]
  [  0.   0.   0.   0.]
  [  0.   0.   0.   0.]
  [  0.   0.   0.   0.]]]
[[ True  True  True False False False False False]
 [ True  True  True False False False False False]
 [ True  True  True False False  True False False]]

    gt_scores  (3, 8, 1)
[[[1.]

In [71]:
gt_array        = tf.concat([bbox_idx, gt_scores_exp , gt_bboxes, gt_classes_exp], axis=2)
print('    gt_array  ',  type(gt_array), gt_array.shape)
print(gt_array.eval())


    gt_array   <class 'tensorflow.python.framework.ops.Tensor'> (3, 8, 7)
[[[  0.   1.  96.  60.  55. 116.   1.]
  [  1.   1. 116.  95.  45.  99.   2.]
  [  2.   1. 119.  18.  61. 104.   3.]
  [  0.   0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.   0.]]

 [[  0.   1.  69.  61.  34.  33.   3.]
  [  1.   1. 106.  82.  38.  77.   3.]
  [  2.   1.  36.   4. 115.  77.   3.]
  [  0.   0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.   0.]]

 [[  0.   1.   1. 115.  90.  29.   1.]
  [  1.   1.  75.  66.  49.  56.   2.]
  [  2.   1.  95.  91.  21. 114.   2.]
  [  0.   0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.   0.]
  [  5.   1.   0.   0.   0.   0.   2.]
  [  0.   0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   

In [72]:
scatter_classes = tf.stack([batch_grid , gt_class_ids, bbox_grid],axis = -1)
print('\n    -- stack results ----')
print('\n    scatter_classes', type(scatter_classes), 'shape',tf.shape(scatter_classes).eval())
print(scatter_classes.eval())


    -- stack results ----

    scatter_classes <class 'tensorflow.python.framework.ops.Tensor'> shape [3 8 3]
[[[0 1 0]
  [0 2 1]
  [0 3 2]
  [0 0 3]
  [0 0 4]
  [0 0 5]
  [0 0 6]
  [0 0 7]]

 [[1 3 0]
  [1 3 1]
  [1 3 2]
  [1 0 3]
  [1 0 4]
  [1 0 5]
  [1 0 6]
  [1 0 7]]

 [[2 1 0]
  [2 2 1]
  [2 2 2]
  [2 0 3]
  [2 0 4]
  [2 2 5]
  [2 0 6]
  [2 0 7]]]


In [63]:
gt_scatter = tf.scatter_nd(scatter_classes, gt_array, [batch_size, num_classes, num_detections,7])
print('    gt_tensor shape is ', gt_scatter.get_shape(), gt_scatter)
gt_scatter.eval()

    gt_tensor shape is  (3, 4, 8, 7) Tensor("ScatterNd_2:0", shape=(3, 4, 8, 7), dtype=float32)


array([[[[  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.]],

        [[  0.,   1.,  47., 102.,  91.,  30.,   1.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.]],

        [[  0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  1.,   1.,  54., 109., 100.,  68.,   2.],
         [  0.,   0.,   0.,   0.,   0.,   

In [77]:
sort_vals, sort_inds = tf.nn.top_k(gt_scatter[:,:,:,0], k=gt_scatter.shape[2])
print('    sort vals shape : ', sort_vals.get_shape())
print(sort_vals.eval())
print('    sort inds shape : ', sort_inds.get_shape())
print(sort_inds.eval())

    sort vals shape :  (3, 4, 8)
[[[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0.]
  [2. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [2. 1. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [5. 2. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]]
    sort inds shape :  (3, 4, 8)
[[[0 1 2 3 4 5 6 7]
  [0 1 2 3 4 5 6 7]
  [1 0 2 3 4 5 6 7]
  [2 0 1 3 4 5 6 7]]

 [[0 1 2 3 4 5 6 7]
  [0 1 2 3 4 5 6 7]
  [0 1 2 3 4 5 6 7]
  [2 1 0 3 4 5 6 7]]

 [[0 1 2 3 4 5 6 7]
  [0 1 2 3 4 5 6 7]
  [5 2 1 0 3 4 6 7]
  [0 1 2 3 4 5 6 7]]]


In [73]:
# build gathering indexes to use in sorting 
class_grid, batch_grid, bbox_grid = tf.meshgrid(tf.range(num_classes),tf.range(batch_size), tf.range(num_detections))

print('    class_grid  ', type(class_grid) , 'shape', class_grid.get_shape())
print(class_grid.eval())
print('    batch_grid  ', type(batch_grid) , 'shape', batch_grid.get_shape())
print(class_grid.eval())
print('    bbox_grid    ', type(bbox_grid) , 'shape', bbox_grid.get_shape())
print(bbox_grid.eval())

    class_grid   <class 'tensorflow.python.framework.ops.Tensor'> shape (3, 4, 8)
[[[0 0 0 0 0 0 0 0]
  [1 1 1 1 1 1 1 1]
  [2 2 2 2 2 2 2 2]
  [3 3 3 3 3 3 3 3]]

 [[0 0 0 0 0 0 0 0]
  [1 1 1 1 1 1 1 1]
  [2 2 2 2 2 2 2 2]
  [3 3 3 3 3 3 3 3]]

 [[0 0 0 0 0 0 0 0]
  [1 1 1 1 1 1 1 1]
  [2 2 2 2 2 2 2 2]
  [3 3 3 3 3 3 3 3]]]
    batch_grid   <class 'tensorflow.python.framework.ops.Tensor'> shape (3, 4, 8)
[[[0 0 0 0 0 0 0 0]
  [1 1 1 1 1 1 1 1]
  [2 2 2 2 2 2 2 2]
  [3 3 3 3 3 3 3 3]]

 [[0 0 0 0 0 0 0 0]
  [1 1 1 1 1 1 1 1]
  [2 2 2 2 2 2 2 2]
  [3 3 3 3 3 3 3 3]]

 [[0 0 0 0 0 0 0 0]
  [1 1 1 1 1 1 1 1]
  [2 2 2 2 2 2 2 2]
  [3 3 3 3 3 3 3 3]]]
    bbox_grid     <class 'tensorflow.python.framework.ops.Tensor'> shape (3, 4, 8)
[[[0 1 2 3 4 5 6 7]
  [0 1 2 3 4 5 6 7]
  [0 1 2 3 4 5 6 7]
  [0 1 2 3 4 5 6 7]]

 [[0 1 2 3 4 5 6 7]
  [0 1 2 3 4 5 6 7]
  [0 1 2 3 4 5 6 7]
  [0 1 2 3 4 5 6 7]]

 [[0 1 2 3 4 5 6 7]
  [0 1 2 3 4 5 6 7]
  [0 1 2 3 4 5 6 7]
  [0 1 2 3 4 5 6 7]]]


In [75]:
gather_inds = tf.stack([batch_grid , class_grid, sort_inds],axis = -1)
print('    -- pred_tensor results (A-boxes sorted by score ----')
print('    gatehr_inds ', gather_inds.get_shape())
print(gather_inds.eval())

gt_tensor = tf.gather_nd(gt_scatter, gather_inds)
print('    -- pred_tensor results (A-boxes sorted by score ----')
print('    pred_tensor ', gt_tensor.get_shape())
print(gt_tensor.eval())

    -- pred_tensor results (A-boxes sorted by score ----
    gatehr_inds  (3, 4, 8, 3)
[[[[0 0 0]
   [0 0 1]
   [0 0 2]
   [0 0 3]
   [0 0 4]
   [0 0 5]
   [0 0 6]
   [0 0 7]]

  [[0 1 0]
   [0 1 1]
   [0 1 2]
   [0 1 3]
   [0 1 4]
   [0 1 5]
   [0 1 6]
   [0 1 7]]

  [[0 2 1]
   [0 2 0]
   [0 2 2]
   [0 2 3]
   [0 2 4]
   [0 2 5]
   [0 2 6]
   [0 2 7]]

  [[0 3 2]
   [0 3 0]
   [0 3 1]
   [0 3 3]
   [0 3 4]
   [0 3 5]
   [0 3 6]
   [0 3 7]]]


 [[[1 0 0]
   [1 0 1]
   [1 0 2]
   [1 0 3]
   [1 0 4]
   [1 0 5]
   [1 0 6]
   [1 0 7]]

  [[1 1 0]
   [1 1 1]
   [1 1 2]
   [1 1 3]
   [1 1 4]
   [1 1 5]
   [1 1 6]
   [1 1 7]]

  [[1 2 0]
   [1 2 1]
   [1 2 2]
   [1 2 3]
   [1 2 4]
   [1 2 5]
   [1 2 6]
   [1 2 7]]

  [[1 3 2]
   [1 3 1]
   [1 3 0]
   [1 3 3]
   [1 3 4]
   [1 3 5]
   [1 3 6]
   [1 3 7]]]


 [[[2 0 0]
   [2 0 1]
   [2 0 2]
   [2 0 3]
   [2 0 4]
   [2 0 5]
   [2 0 6]
   [2 0 7]]

  [[2 1 0]
   [2 1 1]
   [2 1 2]
   [2 1 3]
   [2 1 4]
   [2 1 5]
   [2 1 6]
   [2 1 7]]

  [[2 2 5

###  Split Box/ Class info for each Image dim(0) by class and place into new tensor using tf.scatter_nd_update
###  obsolete -- use tf.scatter_nd instead
    Output is converted to NUMN_BATCHES x NUM_CLASSES x num_rois x columns 

In [70]:
# scatt1 = tf.scatter_nd(scatter_inds, pred_array, [batch_size, num_classes, num_rois])
# print('scatt1  shape ', scatt1.get_shape(), scatt)

# zeros = tf.zeros([batch_size,num_classes, num_rois,7], dtype=tf.float32)
# ref   = tf.Variable(zeros)
# init = tf.global_variables_initializer().run()

# pred_scatt_old = tf.scatter_nd_update(ref, scatter_classes, pred_array)


# print('scatter shape is ', pred_scatt.get_shape(), pred_scatt)
# print(pred_scatt[0].eval())
# print('------------')
# print(pred_scatt[1].eval())
# print('------------')
# print(pred_scatt[2].eval())

